## PROYECTO 1 - MINERIA DE DATOS

In [2]:
# Importando librerias necesarias
import pandas as pd

In [3]:
# Cargando el data
data_2011 = pd.read_spss('matrimonio2011.sav')
data_2011 = pd.DataFrame(data_2011)

data_2012 = pd.read_spss('matrimonio2012.sav')
data_2012 = pd.DataFrame(data_2012)

data_2013 = pd.read_spss('matrimonio2013.sav')
data_2013 = pd.DataFrame(data_2013)

data_2014 = pd.read_spss('matrimonio2014.sav')
data_2014 = pd.DataFrame(data_2014)

data_2015 = pd.read_spss('matrimonio2015.sav')
data_2015 = pd.DataFrame(data_2015)

data_2016 = pd.read_spss('matrimonio2016.sav')
data_2016 = pd.DataFrame(data_2016)

data_2017 = pd.read_spss('matrimonio2017.sav')
data_2017 = pd.DataFrame(data_2017)

data_2018 = pd.read_spss('matrimonio2018.sav')
data_2018 = pd.DataFrame(data_2018)

data_2019 = pd.read_spss('matrimonio2019.sav')
data_2019 = pd.DataFrame(data_2019)

data_2020 = pd.read_spss('matrimonio2020.sav')
data_2020 = pd.DataFrame(data_2020)

data_2021 = pd.read_spss('matrimonio2021.sav')
data_2021 = pd.DataFrame(data_2021)


In [108]:
# Limpieza data - 2011

columns_use_2011 = []

# Clase de union en matrimonio
union_tipos_2011 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2011['claseUnion'] = data_2011['Clauni'].apply(lambda x: union_tipos_2011.get(x, 0))
columns_use_2011.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2011 = {'No indigena': 1, 'Indigena': 2}
data_2011['grupoEtnicoHombre'] = data_2011['Gethom'].apply(lambda x: grupos_etnicos_2011.get(x, 0))
data_2011['grupoEtnicoMujer'] = data_2011['Getmuj'].apply(lambda x: grupos_etnicos_2011.get(x, 0))
columns_use_2011.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre 
data_2011['edadHombre'] = data_2011['Edadhom']
columns_use_2011.append('edadHombre')

# Edad de la mujer
data_2011['edadMujer'] = data_2011['Edadmuj']
columns_use_2011.append('edadMujer')


In [109]:
# Limpieza data - 2012

columns_use_2012 = []

# Clase de union en matrimonio
union_tipos_2012 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2012['claseUnion'] = data_2012['CLAUNI'].apply(lambda x: union_tipos_2012.get(x, 0))
columns_use_2012.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2012 = {'No indigena': 1, 'Indigena': 2}
data_2012['grupoEtnicoHombre'] = data_2012['GETHOM'].apply(lambda x: grupos_etnicos_2012.get(x, 0))
data_2012['grupoEtnicoMujer'] = data_2012['GETMUJ'].apply(lambda x: grupos_etnicos_2012.get(x, 0))
columns_use_2012.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2012['edadHombre'] = data_2012['EDADHOM']
columns_use_2012.append('edadHombre')

# Edad de la mujer
data_2012['edadMujer'] = data_2012['EDADMUJ']
columns_use_2012.append('edadMujer')


In [110]:
# Limpieza data - 2013

columns_use_2013 = []

# Clase de union en matrimonio
union_tipos_2013 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2013['claseUnion'] = data_2013['CLAUNI'].apply(lambda x: union_tipos_2013.get(x, 0))
columns_use_2013.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2013 = {'Mestizo / Ladino': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2013['grupoEtnicoHombre'] = data_2013['PUEHOM'].apply(lambda x: grupos_etnicos_2013.get(x, 0))
data_2013['grupoEtnicoMujer'] = data_2013['PUEMUJ'].apply(lambda x: grupos_etnicos_2013.get(x, 0))
columns_use_2013.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2013['edadHombre'] = data_2013['EDADHOM']
columns_use_2013.append('edadHombre')

# Edad de la mujer
data_2013['edadMujer'] = data_2013['EDADMUJ']
columns_use_2013.append('edadMujer')



In [113]:
# Limpieza data - 2014

columns_use_2014 = []

# Clase de union en matrimonio
union_tipos_2014 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2014['claseUnion'] = data_2014['CLAUNI'].apply(lambda x: union_tipos_2014.get(x, 0))
columns_use_2014.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2014 = {'Mestizo / Ladino': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2014['grupoEtnicoHombre'] = data_2014['PUEHOM'].apply(lambda x: grupos_etnicos_2014.get(x, 0))
data_2014['grupoEtnicoMujer'] = data_2014['PUEMUJ'].apply(lambda x: grupos_etnicos_2014.get(x, 0))
columns_use_2014.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2014['edadHombre'] = data_2014['EDADHOM']
columns_use_2014.append('edadHombre')

# Edad de la mujer
data_2014['edadMujer'] = data_2014['EDADMUJ']
columns_use_2014.append('edadMujer')


In [114]:
# Limpieza data - 2015

columns_use_2015 = []

# Clase de union en matrimonio
union_tipos_2015 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2015['claseUnion'] = data_2015['CLAUNI'].apply(lambda x: union_tipos_2015.get(x, 0))
columns_use_2015.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2015 = {'Mestizo / Ladino': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2015['grupoEtnicoHombre'] = data_2015['PUEHOM'].apply(lambda x: grupos_etnicos_2015.get(x, 0))
data_2015['grupoEtnicoMujer'] = data_2015['PUEMUJ'].apply(lambda x: grupos_etnicos_2015.get(x, 0))
columns_use_2015.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2015['edadHombre'] = data_2015['EDADHOM']
columns_use_2015.append('edadHombre')

# Edad de la mujer
data_2015['edadMujer'] = data_2015['EDADMUJ']
columns_use_2015.append('edadMujer')


In [115]:
# Limpieza data - 2016

columns_use_2016 = []

# Clase de union en matrimonio
union_tipos_2016 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2016['claseUnion'] = data_2016['CLAUNI'].apply(lambda x: union_tipos_2016.get(x, 0))
columns_use_2016.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2016 = {'Mestizo / Ladino': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2016['grupoEtnicoHombre'] = data_2016['PUEHOM'].apply(lambda x: grupos_etnicos_2016.get(x, 0))
data_2016['grupoEtnicoMujer'] = data_2016['PUEMUJ'].apply(lambda x: grupos_etnicos_2016.get(x, 0))
columns_use_2016.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2016['edadHombre'] = data_2016['EDADHOM']
columns_use_2016.append('edadHombre')

# Edad de la mujer
data_2016['edadMujer'] = data_2016['EDADMUJ']
columns_use_2016.append('edadMujer')


In [116]:
# Limpieza data - 2017

columns_use_2017 = []

# Clase de union en matrimonio
union_tipos_2017 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2017['claseUnion'] = data_2017['CLAUNI'].apply(lambda x: union_tipos_2017.get(x, 0))
columns_use_2017.append('claseUnion')

# Grupo etnico de hombre
grupos_etnicos_hombre_2017 = {'Ladino / Mestizo': 1, 'Maya': 2, 'Garífuna': 2, 'Xinca': 2}
data_2017['grupoEtnicoHombre'] = data_2017['PUEHOM'].apply(lambda x: grupos_etnicos_hombre_2017.get(x, 0))
columns_use_2017.append('grupoEtnicoHombre')

# Grupo etnico de mujer
grupos_etnicos_mujer_2017 = {'Mestizo / Ladino': 1, 'Maya': 2, 'Garífuna': 2, 'Xinca': 2}
data_2017['grupoEtnicoMujer'] = data_2017['PUEMUJ'].apply(lambda x: grupos_etnicos_mujer_2017.get(x, 0))
columns_use_2017.append('grupoEtnicoMujer')

# Edad del hombre
data_2017['edadHombre'] = data_2017['EDADHOM']
columns_use_2017.append('edadHombre')

# Edad de la mujer
data_2017['edadMujer'] = data_2017['EDADMUJ']
columns_use_2017.append('edadMujer')


In [117]:
# Limpieza data - 2018

columns_use_2018 = []

# Clase de union en matrimonio
union_tipos_2018 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2018['claseUnion'] = data_2018['CLAUNI'].apply(lambda x: union_tipos_2018.get(x, 0))
columns_use_2018.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2018 = {'Ladino / Mestizo': 1, 'Maya': 2, 'Garífuna': 2, 'Xinca': 2}
data_2018['grupoEtnicoHombre'] = data_2018['PUEHOM'].apply(lambda x: grupos_etnicos_2018.get(x, 0))
data_2018['grupoEtnicoMujer'] = data_2018['PUEMUJ'].apply(lambda x: grupos_etnicos_2018.get(x, 0))
columns_use_2018.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2018['edadHombre'] = data_2018['EDADHOM']
columns_use_2018.append('edadHombre')

# Edad de la mujer
data_2018['edadMujer'] = data_2018['EDADMUJ']
columns_use_2018.append('edadMujer')


In [118]:
# Limpieza data - 2019

columns_use_2019 = []

# Clase de union en matrimonio
union_tipos_2019 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2019['claseUnion'] = data_2019['CLAUNI'].apply(lambda x: union_tipos_2019.get(x, 0))
columns_use_2019.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2019 = {'Ladino / Mestizo': 1, 'Maya': 2, 'Garífuna': 2, 'Xinca': 2}
data_2019['grupoEtnicoHombre'] = data_2019['PUEHOM'].apply(lambda x: grupos_etnicos_2019.get(x, 0))
data_2019['grupoEtnicoMujer'] = data_2019['PUEMUJ'].apply(lambda x: grupos_etnicos_2019.get(x, 0))
columns_use_2019.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2019['edadHombre'] = data_2019['EDADHOM']
columns_use_2019.append('edadHombre')

# Edad de la mujer
data_2019['edadMujer'] = data_2019['EDADMUJ']
columns_use_2019.append('edadMujer')


In [119]:
# Limpieza data - 2020

columns_use_2020 = []

# Clase de union en matrimonio
union_tipos_2020 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2020['claseUnion'] = data_2020['CLAUNI'].apply(lambda x: union_tipos_2020.get(x, 0))
columns_use_2020.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2020 = {'Ladino / Mestizo': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2020['grupoEtnicoHombre'] = data_2020['PUEHOM'].apply(lambda x: grupos_etnicos_2020.get(x, 0))
data_2020['grupoEtnicoMujer'] = data_2020['PUEMUJ'].apply(lambda x: grupos_etnicos_2020.get(x, 0))
columns_use_2020.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2020['edadHombre'] = data_2020['EDADHOM']

# Edad de la mujer
data_2020['edadMujer'] = data_2020['EDADMUJ']
columns_use_2020.append('edadMujer')


In [99]:
# Limpieza data - 2021

columns_use_2021 = []

# Clase de union en matrimonio
union_tipos_2021 = {'Comunidad de gananciales': 1, 'Comunidad absoluta': 2, 'Separación absoluta': 3}
data_2021['claseUnion'] = data_2021['CLAUNI'].apply(lambda x: union_tipos_2021.get(x, 0))
columns_use_2021.append('claseUnion')

# Grupo etnico de hombre y mujer
grupos_etnicos_2021 = {'Ladino / Mestizo': 1, 'Maya': 2, 'Garífuna': 2, 'Xinka': 2}
data_2021['grupoEtnicoHombre'] = data_2021['PUEHOM'].apply(lambda x: grupos_etnicos_2021.get(x, 0))
data_2021['grupoEtnicoMujer'] = data_2021['PUEMUJ'].apply(lambda x: grupos_etnicos_2021.get(x, 0))
columns_use_2021.extend(['grupoEtnicoMujer', 'grupoEtnicoHombre'])

# Edad del hombre
data_2021['edadHombre'] = data_2021['EDADHOM']
columns_use_2021.append('edadHombre')

# Edad de la mujer
data_2021['edadMujer'] = data_2021['EDADMUJ']
columns_use_2021.append('edadMujer')
